## Notebook for testing re-implementation of hamiltonian jacobian calculation

In [1]:
import pygsti
from pygsti.extras.idletomography.idtcore import hamiltonian_jac_alt, build_class_jacobian
from pygsti.extras.idletomography.pauliobjs import NQPauliState
import numpy as np

simplest test, one-qubit hamiltonian jacobian

In [2]:
ham_jac , pairs=  hamiltonian_jac_alt(1)

Compare against existing code as baseline. I think in the existing code the function `build_class_jacobian` constructs the jacobian for all of the prep-measure pairs, and then this gets used to downselect to a subset of pairs, this should be what we want to compare against here.

In [3]:
diff_pairs = [pair for pair in pairs if pair[0][-1] != pair[1][-1]]

In [4]:
ham_jac_orig = build_class_jacobian('H', 1)
#refactor this to match output of new code
ham_jac_orig_refactored_dict = {}
for (key, val) in ham_jac_orig.items():
    if key[1] =='X':
        ham_jac_orig_refactored_dict[key[0]] = [val, 0, 0]
    if key[1] =='Y':
        ham_jac_orig_refactored_dict[key[0]] = [0, val, 0]
    if key[1] =='Z':
        ham_jac_orig_refactored_dict[key[0]] = [0, 0, val]

ham_jac_orig_refactored = np.array( for pair in diff_pairs])
print(ham_jac_orig_refactored_dict)

4
{('+X', '+Z'): [0, -2.0, 0], ('-X', '+Z'): [0, 2.0, 0], ('+X', '+Y'): [0, 0, 2.0], ('-X', '+Y'): [0, 0, -2.0], ('+Y', '+Z'): [2.0, 0, 0], ('-Y', '+Z'): [-2.0, 0, 0], ('+Y', '+X'): [0, 0, -2.0], ('-Y', '+X'): [0, 0, 2.0], ('+Z', '+Y'): [-2.0, 0, 0], ('-Z', '+Y'): [2.0, 0, 0], ('+Z', '+X'): [0, 2.0, 0], ('-Z', '+X'): [0, -2.0, 0]}


In [5]:
j=0
for i in range(len(pairs)):
    #only print the ones where the prep and meas are different.
    if pairs[i][0][-1] == pairs[i][1][-1]:
        continue
    print(f'{pairs[i]}  {ham_jac[i,:]}  {ham_jac_orig_refactored[j, :]}')
    j+=1

('+X', '+Y')  [0. 0. 2.]  [0. 0. 2.]
('+X', '+Z')  [ 0. -2.  0.]  [ 0. -2.  0.]
('-X', '+Y')  [ 0.  0. -2.]  [ 0.  0. -2.]
('-X', '+Z')  [0. 2. 0.]  [0. 2. 0.]
('+Y', '+X')  [ 0.  0. -2.]  [ 0.  0. -2.]
('+Y', '+Z')  [2. 0. 0.]  [2. 0. 0.]
('-Y', '+X')  [0. 0. 2.]  [0. 0. 2.]
('-Y', '+Z')  [-2.  0.  0.]  [-2.  0.  0.]
('+Z', '+X')  [0. 2. 0.]  [0. 2. 0.]
('+Z', '+Y')  [-2.  0.  0.]  [-2.  0.  0.]
('-Z', '+X')  [ 0. -2.  0.]  [ 0. -2.  0.]
('-Z', '+Y')  [2. 0. 0.]  [2. 0. 0.]


Spot checking these the corresponding pairs do look to match up as expected.

In [6]:
str(NQPauliState("X", (1,))).strip

<function str.strip(chars=None, /)>

In [7]:
"+X+X-Y".replace('+', '').replace('-','')

'XXY'

In [8]:
prep = NQPauliState("XY", (1,-1))
meas = NQPauliState("XZ", (-1,-1))

stripped_prep = str(prep).replace('+', '').replace('-', '')
stripped_meas = str(meas).replace('+', '').replace('-', '')

#calculate the overall parity
overall_sign_prep = np.prod(prep.signs)
overall_sign_meas = np.prod(meas.signs)

overall_signed_pauli_prep = '+' + stripped_prep if overall_sign_prep>0 else '-' + stripped_prep
overall_signed_pauli_meas = '+' + stripped_meas if overall_sign_meas>0 else '-' + stripped_meas

In [9]:
overall_signed_pauli_prep

'-XY'

In [10]:
overall_signed_pauli_meas

'+XZ'

Try the new hamiltonian code now with a set of prep and measure pairs specified.

In [11]:
fid_pairs = [
    (NQPauliState("X", (1,)), NQPauliState("X", (1,))),
    (NQPauliState("X", (1,)), NQPauliState("Y", (1,))),
    (NQPauliState("X", (1,)), NQPauliState("Z", (1,))),
    (NQPauliState("Y", (1,)), NQPauliState("X", (1,))),
    (NQPauliState("Y", (1,)), NQPauliState("Y", (1,))),
    (NQPauliState("Y", (1,)), NQPauliState("Z", (1,))),
    (NQPauliState("Z", (1,)), NQPauliState("X", (1,))),
    (NQPauliState("Z", (1,)), NQPauliState("Y", (1,))),
    (NQPauliState("Z", (1,)), NQPauliState("Z", (1,))),
    (NQPauliState("X", (-1,)), NQPauliState("X", (1,))),
    (NQPauliState("X", (-1,)), NQPauliState("Y", (1,))),
    (NQPauliState("X", (-1,)), NQPauliState("Z", (1,))),
    (NQPauliState("Y", (-1,)), NQPauliState("X", (1,))),
    (NQPauliState("Y", (-1,)), NQPauliState("Y", (1,))),
    (NQPauliState("Y", (-1,)), NQPauliState("Z", (1,))),
    (NQPauliState("Z", (-1,)), NQPauliState("X", (1,))),
    (NQPauliState("Z", (-1,)), NQPauliState("Y", (1,))),
    (NQPauliState("Z", (-1,)), NQPauliState("Z", (1,))),
]

In [12]:
ham_jac_specific_fid_pairs, pairs = hamiltonian_jac_alt(1, prep_meas_fid_pairs=fid_pairs)

In [13]:
print(pairs)

[('+X', '+X'), ('+X', '+Y'), ('+X', '+Z'), ('+Y', '+X'), ('+Y', '+Y'), ('+Y', '+Z'), ('+Z', '+X'), ('+Z', '+Y'), ('+Z', '+Z'), ('-X', '+X'), ('-X', '+Y'), ('-X', '+Z'), ('-Y', '+X'), ('-Y', '+Y'), ('-Y', '+Z'), ('-Z', '+X'), ('-Z', '+Y'), ('-Z', '+Z')]


In [19]:
j=0
for i, pair in enumerate(pairs):
    #only print the ones where the prep and meas are different.
    if pair[0][-1] == pair[1][-1]:
        continue
    print(f'{pairs[i]}  {ham_jac_specific_fid_pairs[i,:]}  {ham_jac_orig_refactored_dict[pair]}')
    j+=1

('+X', '+Y')  [0. 0. 2.]  [0, 0, 2.0]
('+X', '+Z')  [ 0. -2.  0.]  [0, -2.0, 0]
('+Y', '+X')  [ 0.  0. -2.]  [0, 0, -2.0]
('+Y', '+Z')  [2. 0. 0.]  [2.0, 0, 0]
('+Z', '+X')  [0. 2. 0.]  [0, 2.0, 0]
('+Z', '+Y')  [-2.  0.  0.]  [-2.0, 0, 0]
('-X', '+Y')  [ 0.  0. -2.]  [0, 0, -2.0]
('-X', '+Z')  [0. 2. 0.]  [0, 2.0, 0]
('-Y', '+X')  [0. 0. 2.]  [0, 0, 2.0]
('-Y', '+Z')  [-2.  0.  0.]  [-2.0, 0, 0]
('-Z', '+X')  [ 0. -2.  0.]  [0, -2.0, 0]
('-Z', '+Y')  [2. 0. 0.]  [2.0, 0, 0]


Ok, so this seems to be working as well.